# Move WISTAR's CDS Files To a Temporary Staging Folder
## Date: 2022-02-24
## Author: Jeffrey Grover
**Purpose:** Do the same as with HCI's files in support of the CDS data submission.

### Load libraries

In [1]:
library(tidyverse)
library(readxl)
library(sevenbridges)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### Load the metadata

In [2]:
# Load the files for cds upload

cds_files <- read_excel('wistar_files_sampleattributes_DS.xlsx')$FILENAME

length(cds_files)
head(cds_files)

[1] 370

[1] "wm3907n_R1.fastq.gz" "wm3907n_R2.fastq.gz" "wm3922n_R1.fastq.gz"
[4] "wm3922n_R2.fastq.gz" "wm3926n_R1.fastq.gz" "wm3926n_R2.fastq.gz"

In [3]:
# Load the cgc metadata

cgc_files <- read_csv('2022-02-24_wistar_rnaseq_wes_manifest_20220224_095810.csv')

head(cgc_files)

Rows: 382 Columns: 66

── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (30): id, name, project, p1_1, passage, sample_id, sample_type, case_id,...
dbl  (3): size, paired_end, update_manifest
lgl (33): trimmed_read_count, Is FFPE, prop_ribosomal_bases, library_prep_ki...


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



id,name,size,project,trimmed_read_count,p1_1,Is FFPE,passage,sample_id,prop_ribosomal_bases,⋯,dbgap_upload,model_id,paired_end,__inherit__,update_manifest,species,tumor_id,investigation,prop_20x_cov,experimental_strategy
<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<lgl>,⋯,<lgl>,<lgl>,<dbl>,<lgl>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
5d805c02e4b06d0cb47c4dde,WISTAR/WES/wm4281_R1.fastq.gz,10388246367,pdxnet/pdxnet-datapool,NA,NA,NA,tumor,wm4281,NA,⋯,NA,NA,1,NA,NA,NA,NA,NA,NA,WES
5d805c02e4b06d0cb47c4ecf,WISTAR/WES/wm4281_R2.fastq.gz,11865334115,pdxnet/pdxnet-datapool,NA,NA,NA,tumor,wm4281,NA,⋯,NA,NA,2,NA,NA,NA,NA,NA,NA,WES
5d805c02e4b06d0cb47c4f19,WISTAR/WES/wm4281n_R1.fastq.gz,2742803646,pdxnet/pdxnet-datapool,NA,NA,NA,blood,wm4281n,NA,⋯,NA,NA,1,NA,NA,NA,NA,NA,NA,WES
5d805c02e4b06d0cb47c4f82,WISTAR/WES/wm4281n_R2.fastq.gz,3256676515,pdxnet/pdxnet-datapool,NA,NA,NA,blood,wm4281n,NA,⋯,NA,NA,2,NA,NA,NA,NA,NA,NA,WES
5d805c02e4b06d0cb47c4e1e,WISTAR/WES/wm4304_R1.fastq.gz,8838299944,pdxnet/pdxnet-datapool,NA,NA,NA,tumor,wm4304,NA,⋯,NA,NA,1,NA,NA,NA,NA,NA,NA,WES
5d805c02e4b06d0cb47c4e23,WISTAR/WES/wm4304_R2.fastq.gz,10346466097,pdxnet/pdxnet-datapool,NA,NA,NA,tumor,wm4304,NA,⋯,NA,NA,2,NA,NA,NA,NA,NA,NA,WES


In [4]:
# I don't need all the column, so just get the filename and id from the cgc files

cgc_files <- cgc_files %>% select(id, name)

# basename for the name field

cgc_files <- cgc_files %>% mutate(name = basename(name))

head(cgc_files)
nrow(cgc_files)

id,name
<chr>,<chr>
5d805c02e4b06d0cb47c4dde,wm4281_R1.fastq.gz
5d805c02e4b06d0cb47c4ecf,wm4281_R2.fastq.gz
5d805c02e4b06d0cb47c4f19,wm4281n_R1.fastq.gz
5d805c02e4b06d0cb47c4f82,wm4281n_R2.fastq.gz
5d805c02e4b06d0cb47c4e1e,wm4304_R1.fastq.gz
5d805c02e4b06d0cb47c4e23,wm4304_R2.fastq.gz


[1] 382

### Check the files and filter for only cds files
Are all the cds files contained in the cgc files? If so, I can create a manifest that only has the cds files in it.

In [5]:
all(cds_files %in% cgc_files$name)

[1] TRUE

In [6]:
# Filter the cgc files for only the cds files

cds_files_manifest <- cgc_files %>% filter(name %in% cds_files)

nrow(cds_files_manifest)
head(cds_files_manifest)

[1] 370

id,name
<chr>,<chr>
5d805c02e4b06d0cb47c4dde,wm4281_R1.fastq.gz
5d805c02e4b06d0cb47c4ecf,wm4281_R2.fastq.gz
5d805c02e4b06d0cb47c4f19,wm4281n_R1.fastq.gz
5d805c02e4b06d0cb47c4f82,wm4281n_R2.fastq.gz
5d805c02e4b06d0cb47c4e1e,wm4304_R1.fastq.gz
5d805c02e4b06d0cb47c4e23,wm4304_R2.fastq.gz


### Copy the files to the cds staging folder

In [7]:
# Connect to the platform

sbg_auth <- Auth(from = 'file', profile_name = 'cgc')
sbg_proj <- sbg_auth$project(id = 'pdxnet/pdxnet-datapool')

Authenticating with user configuration file: ~/.sevenbridges/credentials

Authenticating with user profile: cgc



In [8]:
# Get the folder object for the staging area

staging_folder <- sbg_proj$get_root_folder()$list_folder_contents(complete = TRUE)[[11]]$list_folder_contents(complete = TRUE)[[1]]

staging_folder
staging_folder$id

== Files ==
id : 6217a99bb07915019d2c0d29
name : 2022-02-24_cds_upload_staging
project : pdxnet/pdxnet-datapool
parent : 5d60008fe4b0b892538d34b4
type : folder

[1] "6217a99bb07915019d2c0d29"

In [9]:
# Copy files to the staging area

for (i in seq_along(cds_files_manifest$id)) {
    
    # Pause for 5 mins every 200 iterations to avoid API call limits
    if (i %% 200 == 0) {
        Sys.sleep(300)
    }
    
    # Copy files
    sbg_proj$file(id = cds_files_manifest$id[i])$copy_to_folder(staging_folder$id)
}

### Set the metadata for these files after the upload
The CDS upload was successful for all of these files so we can go ahead and add the `cds_status` and `dbgap_accession` metadata fields to these files.

In [10]:
for (i in seq_along(cds_files_manifest$id)) {
    
    # Pause for 5 mins every 200 iterations to avoid API call limits
    if (i %% 200 == 0) {
        Sys.sleep(300)
    }
    
    # Set metadata
    sbg_proj$file(id = cds_files_manifest$id[i])$set_meta(cds_status = 'uploaded', dbgap_accession = 'phs002432.v1.p1')
}